In [1]:
import antares

Loading ANTARES from /data0/sw/antares-kernel/lib/python3.9/site-packages/antares/__init__.py

        _    _   _ _____  _    ____  _____ ____
       / \  | \ | |_   _|/ \  |  _ \| ____/ ___|
      / _ \ |  \| | | | / _ \ | |_| |  _| \___ \\
     / ___ \| |\  | | |/ ___ \|  _ /| |___ ___| |
    /_/   \_\_| \_| |_/_/   \_\_| \_\_____|____/   v2.11.0
    


In [2]:
__nbid__ = '0062'
__author__ = 'Carl Stubens, Sebastián Vicencio'
__edited__ = 'Gautham Narayan, Chien-Hsiu Lee, ANTARES Team <antares@noirlab.edu>'
__version__ = '20240601' # yyyymmdd
__datasets__ = ['']
__keywords__ = ['ANTARES', 'transient']

In [3]:
# Imports
import antares.devkit as dk
dk.init()
# You should see a happy message that says that "ANTARES DevKit is ready!"

Jaeger tracer already initialized, skipping


2025-03-10 08:13:08,457 - WARNING MainThread settings.py:setup_prometheus:124 - Prometheus failed to start with [Errno 98] Address already in use
Testing loading a random Locus with `dk.get_locus()`...

ANTARES v2.11.0 DevKit is ready!
Website: https://antares.noirlab.edu
Documentation: https://nsf-noirlab.gitlab.io/csdc/antares/antares/



# Writing a preliminary microlensing filter that reads in the photometry

In [33]:
from statsmodels.stats.weightstats import DescrStatsW
import numpy as np
from astropy.table import MaskedColumn
import warnings
import astropy
import matplotlib.pyplot as plt

# Microlensing detection code is partly from 

class microlensing(dk.Filter):    
    INPUT_LOCUS_PROPERTIES = [
        'ztf_object_id',
    ]
    
    OUTPUT_TAGS = [
        {
            'name': 'microlensing_candidate',
            'description': 'Locus - a transient candidate - exhibits a microlensing-like variability',
        }
    ]


    def make_lc(self, locus):

        with warnings.catch_warnings():
            # The cast of locus.timeseries: astropy.table.Table to a pandas
            # dataframe results in the conversion of some integer-valued
            # columns to floating point represntation. This can result in a
            # number of noisy warning so we will catch & ignore them for the
            # next couple of lines.
            warnings.simplefilter("ignore", astropy.table.TableReplaceWarning)
            df = locus.timeseries.to_pandas()

        data = df[['ant_mjd', 'ztf_fid', 'ztf_magpsf', 'ztf_sigmapsf']]
        dn = data.dropna()

        plt.scatter(dn['ant_mjd'][dn['ztf_fid']==1], dn['ztf_magpsf'][dn['ztf_fid']==1], color='g', label='g_band')
        plt.scatter(dn['ant_mjd'][dn['ztf_fid']==2], dn['ztf_magpsf'][dn['ztf_fid']==2], color='r', label='i_band')
        
        plt.xlabel('Time (mjd)')
        plt.ylabel('Magnitude')
        plt.legend()


    def is_microlensing_candidate(self, times, mags, errors):
        """
        Efficient microlensing detection using heuristics & template matching.
        """
        if len(times) < 10:  # Too few data points
            return False

        # Sort data by time
        sorted_idx = np.argsort(times)
        times, mags, errors = times[sorted_idx], mags[sorted_idx], errors[sorted_idx]

        # 1. Check for smoothness (low skewness means symmetric light curve)
        if abs(skew(mags)) > 1:
            return False

        # 2. Check for achromaticity (g-band vs i-band similarity)
        if np.std(mags) / np.mean(mags) > 0.2:  # Reject if color variation is large
            return False

        # 3. Check variability (microlensing should have a clear peak)
        if np.ptp(mags) < 0.5:  # Peak-to-peak magnitude difference
            return False

        # 4. Perform a lightweight template fit (Paczyński model)
        t0_guess = times[np.argmin(mags)]  # Min mag is peak time
        initial_guess = [t0_guess, 0.1, 20, np.median(mags), 0.1]  # Initial params

        try:
            popt, _ = curve_fit(paczynski, times, mags, p0=initial_guess, sigma=errors, maxfev=5000)
            chi2 = np.sum(((mags - paczynski(times, *popt)) / errors) ** 2) / len(times)

            # 5. Apply a simple chi2 threshold
            if chi2 < 2:  # Well-fit light curves pass
                return True
        except RuntimeError:
            return False  # Fit failed

        return False
    def run(self, locus):
        print('Processing Locus:', locus.locus_id)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", astropy.table.TableReplaceWarning)
            df = locus.timeseries.to_pandas()

        data = df[['ant_mjd', 'ztf_fid', 'ztf_magpsf', 'ztf_sigmapsf']].dropna()

        # Split into g-band and i-band
        for band in [1, 2]:  # 1 = g-band, 2 = i-band
            band_data = data[data['ztf_fid'] == band]
            times, mags, errors = band_data['ant_mjd'].values, band_data['ztf_magpsf'].values, band_data['ztf_sigmapsf'].values
            
            if self.is_microlensing_candidate(times, mags, errors):
                print(f'Locus {locus.locus_id} is a microlensing candidate in band {band}')
                return {'microlensing_candidate': True}

        return {'microlensing_candidate': False}

In [34]:
# Fetch 1 random Locus ID from the test dataset
locus_id = dk.get_locus_ids(1)[0]

# Execute the microlensing filter on the locus
report = dk.run_filter(microlensing, locus=locus_id)


Processing Locus: ANT2025bs07tv6e30ui


In [36]:
print(report)

{'locus_id': 'ANT2025bs07tv6e30ui', 'locus_data': FilterContext(locus_id="ANT2025bs07tv6e30ui"), 't': 0.04074877799999754, 'new_locus_properties': {}, 'new_alert_properties': {}, 'new_tags': set(), 'raised_halt': False}
